In [1]:
import setup
setup.init_django()

In [2]:
from market.models import StockQuote

In [8]:
from django.db.models import Avg, F, RowRange, Window
from django.utils import timezone
from datetime import timedelta
from decimal import Decimal

In [13]:
days_ago = 30
now = timezone.now()
start_date = now - timedelta(days=30)
end_date = now

qs = StockQuote.objects.filter(time__range=(start_date, end_date))
qs.count()

12741

In [14]:
total = 0
for obj in qs:
    total += obj.close_price

In [15]:
total / qs.count()

Decimal('370.6656873322345184836355074')

In [16]:
qs.values('company').annotate(avg_price=Avg('close_price'))

<QuerySet [{'company': 3, 'avg_price': Decimal('634.6686316516709512')}, {'company': 5, 'avg_price': Decimal('428.7963671246006390')}, {'company': 2, 'avg_price': Decimal('197.2389634726688103')}, {'company': 1, 'avg_price': Decimal('233.7022529064620832')}]>

In [19]:
count = 5
ticker = "AAPL"
rolling_qs = list(qs.filter(company__ticker=ticker).order_by('-time')[:count])
rolling_qs.reverse()

In [20]:
rolling_qs

[<StockQuote: StockQuote object (2921291)>,
 <StockQuote: StockQuote object (2921292)>,
 <StockQuote: StockQuote object (2921293)>,
 <StockQuote: StockQuote object (2921294)>,
 <StockQuote: StockQuote object (2921295)>]

In [23]:
total = 0
for i, obj in enumerate(rolling_qs):
    total += obj.close_price
    avg = total / (i + 1)
    print(i + 1, obj.id, obj.close_price, avg)

1 2921291 234.6501 234.6501
2 2921292 234.2512 234.45065
3 2921293 234.2700 234.3904333333333333333333333
4 2921294 234.2400 234.352825
5 2921295 233.9200 234.26626


In [24]:
qs = StockQuote.objects.filter(company__ticker=ticker, time__range=(start_date, end_date))

In [27]:
frame_start = -(count - 1)
ma_val = qs.annotate(
        ma=Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame=RowRange(start=frame_start, end=0),
        )
).order_by('-time')

In [30]:
for obj in ma_val[:5]:
    print(obj.id, obj.close_price, obj.ma, obj.time)

2921295 233.9200 234.2662600000000000 2025-02-01 00:55:00+00:00
2921294 234.2400 234.4076800000000000 2025-02-01 00:50:00+00:00
2921293 234.2700 234.5044800000000000 2025-02-01 00:45:00+00:00
2921292 234.2512 234.6104800000000000 2025-02-01 00:40:00+00:00
2921291 234.6501 234.7302400000000000 2025-02-01 00:35:00+00:00


In [29]:
frame_start = -(count - 1)
ma_vals = qs.annotate(
        ma_5=Window(
            expression=Avg('close_price'),
            order_by=F('time').asc(),
            partition_by=[],
            frame=RowRange(start=-4, end=0),
        ),
        ma_20=Window(
                expression=Avg('close_price'),
                order_by=F('time').asc(),
                partition_by=[],
                frame=RowRange(start=-19, end=0),
            )
).order_by('-time').first()

ma_vals.id, ma_vals.close_price, ma_vals.ma_5, ma_vals.ma_20

(2921295,
 Decimal('233.9200'),
 Decimal('234.2662600000000000'),
 Decimal('234.7467450000000000'))